# Inverse Kinematics using Newton's Method

### CS70 — Foundations of Applied Computer Science
---

This notebook contains literate code, i.e. brief fragments of Python surrounded by descriptive text (using Markdown).

## Import statements

In [1]:
import bqplot as bqp
import numpy as np
import scipy.linalg as la

# New: Optimization package
import scipy.optimize as opt

# bqplot plotting library
import bqplot as bqp

# Import graphical user interface components used below
from ipywidgets import interact
from ipywidgets import FloatSlider, VBox


## Part 1: Forward Kinematics

Consider the mathematics of a very simple kind of "skeleton": a chain of bones in two dimensions with joint positions $p_i = (x_i, y_i)$. The first joint is rigidly attached to the origin (i.e. $p_0 = (0, 0)$) while the other joints and bones are free to move in any way. For simplicitly, we also assume that all of the bones have the same length $l_1=l_2=\ldots=1$.

![](images/inverse-kinematics-01.png)

Each parameter $\theta_i\in[0,2\pi]$ specifies the counter-clockwise angle that the associated bone from joint $p_{i-1}$ to joint $p_i$ makes with its predecessor bone (the pair of bones are parallel if $\theta_i=0$). The first bone doesn't have a predecessor, hence $\theta_1$ is measured relative to the $X$ axis. Note how the complete set of bone angles $\theta_1, \theta_2, \ldots$ is all the information we need to compute the precise positions of all the joint positions in Euclidean space.

Forward kinematics (FK) is defined (per lecture) as the problem of converting a set of bone angles $\theta_i$ into joint positions $p_i$. Since $p_i$ depends on all of the preceding angles, we can think of each joint position as a function $p_i=p(\theta_1,\ldots,\theta_{i})$ 




The function `chain_simple` solves the forward kinematics for a chain with at most one bone. The function takes an array of angles as a parameter, which can be of length 0 or 1 (use the Python `len()` function to query the length of an array). When no angles are specified, the function returns the position of the first joint $(x_0,y_0)=(0, 0)$ as an 1D NumPy array. When a single angle is specified, it returns the position $x_1, y_1$.



In [2]:
def chain_simple(theta):
    length = len(theta)
    if length == 0: # return (0,0) if no angles specified
        return np.array([0.,0.])
    angle = theta[0]
    res = np.array([np.cos(angle), np.sin(angle)]) # x,y change for angle
    return res

# TESTS
print('chain_simple:', chain_simple([]))
print('reference:   ', np.array([0.,0.]))
print()
print('chain_simple:', chain_simple([0.]))
print('reference:   ', np.array([1., 0.])) # note all bone lengths are 1
print()
print('chain_simple:', chain_simple([np.pi / 4]))
print('reference:   ', np.array([ 0.70710678,  0.70710678]))


chain_simple: [0. 0.]
reference:    [0. 0.]

chain_simple: [1. 0.]
reference:    [1. 0.]

chain_simple: [0.70710678 0.70710678]
reference:    [0.70710678 0.70710678]


## 1.1 Helper function

The function ``fk_demo()`` below was provided by the TA's to interactively explore the possible chain configurations via forward kinematics. The implementation uses the ``bqplot`` library mentioned above and is fairly technical and beyond the scope of this class.

In [3]:
def fk_demo(chain_func, theta, extra = [[],[]]):
    '''
    This function visualizes the configuration of a chain of bones
    and permits interactive changes to its state. It expects two arguments:
    
    ``chain_func``: a function that implements forward kinematics by
    turning a sequence of angles (theta_1, theta_2, ..., theta_n) into
    the position of the last joint of this chain (x_n, y_n).
    
    ``theta``: an array with the initial angles of all joints
    
    ``extra``: An optional argument which can be used to plot
    additional points that are highlighted in red
    '''
    
    # Function which repeatedly calls ``chain_func`` to compute all joint positions
    def chain_all(theta):
        return np.column_stack([chain_func(theta[:i]) for i in range(0, len(theta) + 1)])

    # Determine size and initial configuration
    size = len(theta)
    positions = chain_all(theta)

    # Define the range of the plotting frame
    scales = { 'x': bqp.LinearScale(min=-size-1, max=size+1),
               'y': bqp.LinearScale(min=-size-1, max=size+1) }

    # Create a scatter plot (for joints), a line plot (for bones), and
    # another scatter plot (to draw extra points specified the ``extra`` argument)
    scat  = bqp.Scatter(scales=scales)
    lines = bqp.Lines(scales=scales)
    scat2 = bqp.Scatter(scales=scales, default_colors=['red'])

    # Create a figure that combines the three plots
    figure = bqp.Figure(marks=[scat, scat2, lines])
    figure.layout.height = '500px'
    figure.layout.width = '500px'

    # Initialize the plots with the initial data
    scat.x, scat.y = positions
    lines.x, lines.y = positions
    scat2.x, scat2.y = extra
    
    sliders = []
    
    # For each angle theta_i,
    for i in range(len(theta)):
        # Create a graphical slider
        slider = FloatSlider(min=0, max=2*np.pi, value=theta[i], step=1e-3)
        
        # Define a callback function that will be triggered when the slider is moved
        def callback(value, i = i):
            theta[i] = value['new']
            positions = chain_all(theta)
            scat.x, scat.y = positions
            lines.x, lines.y = positions

        # "Attach" the callback function to the slider
        slider.observe(callback, 'value')
        sliders.append(slider)

    # Combine the plots and sliders in a vertical arrangement
    return VBox([*sliders, figure])

## 1.2 Visualization of the forward kinematics



We can ensure that the implementation of `chain_simple` satisfies all the specifications by invoking the `fk_demo()` function with arguments `chain_simple` and ``[0.]`` (the initial parameters of a flat chain). We can then drag a slider from 0 to $2\pi$ and see a visual representation of a 1-bone chain turning counter-clockwise.


In [4]:
fk_demo(chain_simple, [0.])


## 1.3 Longer chains



The function `chain` solves the forward kinematics for an arbitrarily long sequence of bones. The function takes an arbitrary-length array of angles as a parameter. When no angles are specified, the function returns the position $(x_0, y_0)$ as before. When $i$ angles are specified, it (only) returns the joint position $(x_{i}, y_{i})$. We use recursion which allows for a particularly simple implementation.


In [5]:
def chain(theta):
    length = len(theta)
    if length == 0: # end case
        return np.array([0.,0.])
    angle = theta[0] # gets current angle
    res = np.array([np.cos(angle), np.sin(angle)]) # x,y change for current angle
    mod_array = theta[1:] # removes processed angle
    if len(mod_array)!=0:
        mod_array[0] += angle # adjusts next angle based on current angle
    return res + chain(mod_array) # recurses through

# TESTS
print('chain:     ', chain([0.1, 0.2, 0.3, 0.4]))
print('reference: ', np.array([ 3.31597858,  1.80146708]))
print()
print('chain:     ', chain([np.pi, np.pi, np.pi, np.pi]))
print('reference: ', np.array([0, 0]))


chain:      [3.31597858 1.80146708]
reference:  [3.31597858 1.80146708]

chain:      [ 0.0000000e+00 -2.4492936e-16]
reference:  [0 0]


## 1.4 Attempting to reach a certain position

Running the command `fk_demo(chain, [0, 0, 0, 0, 0], [[-2], [3]])` below we see a chain with five segments and five corresponding sliders, as well as an additional point highlighted in red.



The following is a configuration of angles that brings the endpoint of the chain as close as possible to the highlighted location `[-2, 3]]`. These parameters were copied into the argment list of the `fk_demo` function call.



In [6]:
fk_demo(chain, [0.80, 0.85, 0.70, 0.38, 0.28], [[-2], [3]])


# Part 2: Inverse Kinematics

Reasoning from Professor: Problems similar to the one in Section 1.4 are tedious to solve by hand: all of the parameters are interdependent and must be adjusted in a coordinated manner. So-called *inverse kinematics* techniques apply numerical root finding to determine solutions to this problem in an automated way. Most modern animation systems have builtin support for inverse kinematics since it allows for a much more convenient workflow: rather than having to tweak each individual bone, artists can directly specify a target shape, and the system will automatically infer all the necessary rotations.

In this part of the project, we will use inverse kinematics to automatically determine $\theta_1,\ldots,\theta_n$ such that

$$
p(\theta_1,\ldots,\theta_n) = p_{\mathrm{target}}
$$

for a given value $p_{\mathrm{target}}\in\mathbb{R}^2$. In other words: the user can move around the endpoint of the chain, and the skeleton will automatically reconfigure itself to follow. This is illustrated in the following figure:

![](images/inverse-kinematics-04.png)

A good numerical root finding technique requires the ability to evaluate the Jacobian of $p$, i.e. all the partial derivatives $\frac{\partial p(\theta_1,\ldots,\theta_n)}{\partial \theta_j}$. The partial derivatives encode how a small perturbation of each of the angles $\theta_j$ leads to a corresponding change in $p(\theta_1,\ldots,\theta_n)$. For simplicity, we'll first look at a 1-segment chain and then derive a solution for the general problem.



We implement a function `dchain_simple(theta)` which takes an array with one entry, and computes the function $\frac{\partial p(\theta_1)}{\partial \theta_1}$. The return value is a two-dimensional array with one column and two rows containing the partial derivatives of the coordinate values $x_1$ and $y_1$. We use analytic methods for the approximation of derivatives.


In [7]:
def dchain_simple(theta):
    par_x = -np.sin(theta[0]) # partial der of x
    par_y = np.cos(theta[0]) # partial der of y
    return np.array([[par_x], [par_y]])


# TESTS
print('dchain_simple: \n', dchain_simple([0]))
print('reference:     \n', np.array([[0.], [1.]]))
print()
print('dchain_simple: \n', dchain_simple([np.pi / 4]))
print('reference:     \n', np.array([[-0.70710678], [ 0.70710678]]))


dchain_simple: 
 [[-0.]
 [ 1.]]
reference:     
 [[0.]
 [1.]]

dchain_simple: 
 [[-0.70710678]
 [ 0.70710678]]
reference:     
 [[-0.70710678]
 [ 0.70710678]]


## 2.1 Implementing the full Jacobian function

Building off the version for a single bone, we'll now implement the full Jacobian $\nabla p(\theta_1, \ldots, \theta_n)$, which is a $2\times n$ matrix containing the partial derivatives with respect to all angles. We use a vector version of the product in our implementation. Specifically, note that

$$
\frac{\partial}{\partial t} \left[A(t)x(t)\right] = A'(t)x(t) + A(t)x'(t)
$$

where $A(t)$ and $x(t)$ are a matrix and a vector depending on a parameter $t$, respectively.



We implement a function `dchain(theta)` which accepts an 1D array of angles with length $\ge 1$ and computes the Jacobian $\nabla p(\theta_1, \ldots, \theta_n)$, a $2\times n$ matrix.


In [8]:
def dchain(theta):
    length = len(theta)
    res = np.empty((2,length)) # result array
    angle_sum = 0
    
    for angle in theta:
        angle_sum += angle # now has total angle
    x_right = 0
    y_right = 0
    
    # fills in result array right to left
    for i in range(length): 
        res[0][length-i-1] = -np.sin(angle_sum) + x_right # dx/d_theta_i (partial)
        res[1][length-i-1] = np.cos(angle_sum) + y_right # dy/d_theta_i (partial)
        angle_sum -= theta[length-i-1] # removes last angle from sum
        x_right = res[0][length-i-1] # caches x element on right (top row)
        y_right = res[1][length-i-1] # caches y element on right (bottom row)
    return res


# TESTS
print('dchain: \n', dchain([0, 0, 0, 0]))
print('reference: \n', np.array([[ 0.,  0.,  0.,  0.], [ 4.,  3.,  2.,  1.]]))
print()
print('dchain: \n', dchain([0.1, 0.2, 0.3]))
print('reference: \n', np.array([[-0.9599961 , -0.86016268, -0.56464247], [ 2.77567627,  1.7806721 ,  0.82533561]]))

dchain: 
 [[0. 0. 0. 0.]
 [4. 3. 2. 1.]]
reference: 
 [[0. 0. 0. 0.]
 [4. 3. 2. 1.]]

dchain: 
 [[-0.9599961  -0.86016268 -0.56464247]
 [ 2.77567627  1.7806721   0.82533561]]
reference: 
 [[-0.9599961  -0.86016268 -0.56464247]
 [ 2.77567627  1.7806721   0.82533561]]


## 2.2 Solving the inverse kinematics problem using Newton's Method

Reason for Newton's method: Newton's method is one of the most widely used methods for finding solutions to systems of non-linear equations. It converges at a remarkable speed when started sufficiently close to a root, though there is generally no strict guarantee of convergence.

Given a function $f(x)$, Newton's method tries to find a solution to the equation $f = 0$ using steps of the form

$$
x_{i+1}=x_i - \left(\nabla f\right)^{-1}f(x_{i}).
$$

In the context of inverse kinematics, we want to apply Newton's method to solve an equation of the form

$$
p(\theta_1,\ldots,\theta_n) = p_{\mathrm{target}}.
$$

for a given reference position $p_{\mathrm{target}}\in\mathbb{R}^2$.

In other words: the unknowns are the angles $\theta_1,\ldots,\theta_n$, and the function whose root we seek maps to a two-dimensional domain. It is not immediately obvious how to apply Newton's method, since the Jacobian of the function has the shape $2\times n$ and hence cannot be inverted using standard techniques like the LU decomposition.

In fact, this is a consequence of the fact that many different configurations can be used to reach the same $p_{\mathrm{target}}$, something evident in part 1.4.

Thus, we must take another approach: *pseudoinverse*. The pseudoinverse is a generalization of the inverse to non-square matrices. In this specific case, the Jacobian is *wide* (i.e. it has more columns than rows), in which case the pseudoinverse will find the solution to a linear system which has the smallest $\|\cdot\|_2$-norm. That is ideal news, since it causes the IK solver to make small adjustments to the angles to reach a new position.


We implement a function `newton(theta, target)` that takes a 1-dimensional array of angles as a starting guess as well as a 2D target position (also specified as a 1-dimensional array) as input. The implementation performs a fixed 8 iterations of Newton's method to try to solve the equation $p(\theta_1,\ldots,\theta_n) = p_{\mathrm{target}}$ and return the final set of parameters $\theta_1,\ldots,\theta_n$ as an 1-dimensional NumPy array. We use the function `la.pinv` to compute the pseudoinverse.


In [9]:
def newton(theta, target):
    for i in range(8): 
        theta_copy = np.copy(theta) # create a copy to avoid modification when passed into chain and dchain, why?
        error = target - chain(theta_copy) # calculate error
        theta = theta + (np.linalg.pinv(dchain(theta_copy))@error) # calculate next theta
    return theta


# TESTS
# Moving a 1-element chain from the default configuration to position (0, 1)
print('chain(newton): ', chain(newton(np.array([0.]), np.array([0., 1.]))))
print('reference:     ', np.array([0, 1]))
print()
# Moving a 2-element chain from the default configuration to position (0.5, 0.5)
print('chain(newton): ', chain(newton(np.array([0., 0.]), np.array([0.5, 0.5]))))
print('reference:     ', np.array([0.5, 0.5]))

chain(newton):  [6.123234e-17 1.000000e+00]
reference:      [0 1]

chain(newton):  [0.4616232  0.46922546]
reference:      [0.5 0.5]


## 2.3 One more helper function

The TA's have provided the function `ik_demo()` below to interactively explore the possible chain configurations via inverse kinematics. Similar to `fk_demo()`, the function is fairly technical and beyond the scope of this class.

In [10]:
def ik_demo(solver, size):    
    theta = np.zeros(size, dtype=np.float64)
    
    # Function which repeatedly calls ``chain`` to compute all joint positions
    def chain_all(theta):
        return np.column_stack([chain(theta[:i]) for i in range(0, len(theta) + 1)])

    # Callback that is invoked when the user drags the red endpoint around
    def refresh(_):
        # 'theta' is a variable of the parent function, we want to modify it here
        nonlocal theta
        
        # Target position
        target = np.array([scat2.x[0], scat2.y[0]])
        
        # Don't try to solve the problem if the user dragged the point out of the circle
        if la.norm(target) > size:
            return
        
        # Call the provided IK solver
        theta = solver(theta, target)
        
        # Update the positions
        values = chain_all(theta)
        scat.x, scat.y = values
        lines.x, lines.y = values
    
    # Similar to fk_solver(), create a number of plots and merge them
    scales = { 'x': bqp.LinearScale(min=-size-1, max=size+1),
               'y': bqp.LinearScale(min=-size-1, max=size+1) }

    scat  = bqp.Scatter(scales=scales)
    lines = bqp.Lines(scales=scales)

    # Create a circle which marks the boundary of where the red point can be moved
    circle_x = np.cos(np.linspace(0, 2*np.pi, 100)) * size
    circle_y = np.sin(np.linspace(0, 2*np.pi, 100)) * size
    circle = bqp.Lines(x=circle_x, y=circle_y,
                       scales=scales, colors=['gray'])
    
    # Special plot, which contains the red endpoint that can be moved
    scat2 = bqp.Scatter(scales=scales,
                        enable_move=True, 
                        update_on_move=True,
                        default_colors=['red'])

    # Initialize the visualizations with the default configuration
    values = chain_all(theta)
    scat.x, scat.y = values
    lines.x, lines.y = values
    scat2.x, scat2.y = chain(theta).reshape(2, 1)
    
    # Call the 'refresh' function when the red dot is moved
    scat2.observe(refresh, names=['x', 'y'])

    figure = bqp.Figure(marks=[scat, scat2, lines, circle])
    figure.layout.height = '500px'
    figure.layout.width = '500px'
    return figure

## 2.4 Visualizing its Behavior

Now to view the behavior of the completed inverse kinematics solver. 
    
The first cell below invokes the IK demonstration with 4 segments, i.e. `ik_demo(newton, 4)`. You can move the red endpoint with your mouse cursor, leading to a smooth adjustment of the chain configuration.

The second cell invokes the IK demonstration with 30 segments, i.e. `ik_demo(newton, 30)`. While the algorithm appears to work, it is noticably slower.

In [11]:
ik_demo(newton, 4)

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='500px', width='500…

In [12]:
ik_demo(newton, 30)

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='500px', width='500…